<a href="https://colab.research.google.com/github/tathagata48/Machine-Learning/blob/main/Car_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [ ]:
df=pd.read_csv('CarPrice_Assignment.csv')
print(df.head(5))
df.shape
df.info()


   car_ID  symboling                   CarName  ... citympg highwaympg    price
0       1          3        alfa-romero giulia  ...      21         27  13495.0
1       2          3       alfa-romero stelvio  ...      21         27  16500.0
2       3          1  alfa-romero Quadrifoglio  ...      19         26  16500.0
3       4          2               audi 100 ls  ...      24         30  13950.0
4       5          2                audi 100ls  ...      18         22  17450.0

[5 rows x 26 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody    

In [ ]:
X = df.drop("price", axis=1)
y = df["price"]


In [ ]:
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(exclude=["object"]).columns

print("Categorical columns:", list(categorical_cols))
print("Numerical columns:", list(numerical_cols))


Categorical columns: ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem']
Numerical columns: ['car_ID', 'symboling', 'wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'boreratio', 'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']


In [ ]:
X_encoded=pd.get_dummies(X,columns=categorical_cols,drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y,train_size=0.2,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((41, 190), (164, 190), (41,), (164,))

In [ ]:
model=LinearRegression()
model.fit(X_train,y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("Train R2:", r2_train)
print("Test R2:", r2_test)
print("Test RMSE:", rmse_test)


Train R2: 1.0
Test R2: 0.731245623316009
Test RMSE: 4000.5078312408646


In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1000.0)  # alpha = regularization strength
ridge.fit(X_train, y_train)

y_pred_test_ridge = ridge.predict(X_test)

from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

r2_test_ridge = r2_score(y_test, y_pred_test_ridge)
rmse_test_ridge = np.sqrt(mean_squared_error(y_test, y_pred_test_ridge))

print("Ridge Test R2:", r2_test_ridge)
print("Ridge Test RMSE:", rmse_test_ridge)


Ridge Test R2: 0.8015422823173696
Ridge Test RMSE: 3437.7267095606558


In [ ]:
from sklearn.linear_model import RidgeCV

alphas = [0.01, 0.1, 1, 10, 100, 1000]

ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(X_train, y_train)

print("Best alpha:", ridge_cv.alpha_)

y_pred_test_cv = ridge_cv.predict(X_test)

from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

print("Test R2:", r2_score(y_test, y_pred_test_cv))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_cv)))


Best alpha: 1000.0
Test R2: 0.8015422823173696
Test RMSE: 3437.7267095606558


In [ ]:
import pandas as pd

coef_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": ridge_cv.coef_
})

coef_df["AbsCoeff"] = coef_df["Coefficient"].abs()
coef_df = coef_df.sort_values(by="AbsCoeff", ascending=False)

coef_df.head(10)


,Feature,Coefficient,AbsCoeff
2,wheelbase,107.517400,107.517400
7,enginesize,67.634563,67.634563
4,carwidth,64.508992,64.508992
3,carlength,53.793226,53.793226
11,horsepower,39.921790,39.921790
13,citympg,-26.346909,26.346909
14,highwaympg,-25.737909,25.737909
179,cylindernumber_six,-24.321012,24.321012
0,car_ID,-23.682638,23.682638
1,symboling,20.393420,20.393420


In [ ]:
# Select a single car from the test set (e.g., the first entry)
car_to_predict = X_test.iloc[[3]]
actual_price = y_test.iloc[3]

# Predict the price for the selected car
predicted_price = ridge_cv.predict(car_to_predict)

print(f"Predicted Price: ${predicted_price[0]:,.2f}")
print(f"Actual Price: ${actual_price:,.2f}")
print(f"Difference: ${abs(predicted_price[0] - actual_price):,.2f}")

Predicted Price: $13,454.81
Actual Price: $11,850.00
Difference: $1,604.81


In [ ]:
import numpy as np
import pandas as pd

# Create a DataFrame for random data, ensuring it has the same columns as X_encoded
# For simplicity, we'll generate random numbers within a plausible range for numerical features
# and randomly select boolean values for the one-hot encoded categorical features.

# Get the columns from the X_encoded DataFrame
model_features = X_encoded.columns

# Create a dictionary to hold random values for each feature
random_data = {}
for col in model_features:
    if X_encoded[col].dtype == bool:
        # For boolean (one-hot encoded) columns, randomly choose True or False
        random_data[col] = [np.random.choice([True, False])]
    elif X_encoded[col].dtype == int or X_encoded[col].dtype == float:
        # For numerical columns, generate a random value within the range observed in the original data
        min_val = X_encoded[col].min()
        max_val = X_encoded[col].max()
        random_data[col] = [np.random.uniform(min_val, max_val)]
    else:
        # Fallback for any other unexpected types (though X_encoded should be numeric/bool)
        random_data[col] = [0] # Default to 0 or another sensible default

random_car_df = pd.DataFrame(random_data, columns=model_features)

# Predict the price for the random car
predicted_price_random = ridge_cv.predict(random_car_df)

print(f"Predicted Price for Random Car: ${predicted_price_random[0]:,.2f}")
print("Random Car Features (first 10):\n", random_car_df.iloc[0].head(26))

Predicted Price for Random Car: $36,587.63
Random Car Features (first 10):
 car_ID                               139.529088
symboling                              1.043003
wheelbase                            103.362611
carlength                            196.936679
carwidth                              68.430546
carheight                              51.48109
curbweight                          4035.169727
enginesize                           321.251272
boreratio                              2.875719
stroke                                 2.809366
compressionratio                      17.629608
horsepower                           209.678854
peakrpm                             4514.089733
citympg                               28.859998
highwaympg                            42.961597
CarName_alfa-romero Quadrifoglio           True
CarName_alfa-romero giulia                False
CarName_alfa-romero stelvio                True
CarName_audi 100 ls                       False
CarName_audi